In [201]:
import fitz
import sys
import pandas as pd
import re

## What is Fitz?

"PyMuPDF is a Python binding for MuPDF – a lightweight PDF, XPS, and E-book viewer, renderer, and toolkit, which is maintained and developed by Artifex Software, Inc" - [PyMu Docs Page](https://pymupdf.readthedocs.io/en/latest/intro.html)


“After Artifex Software acquired the MuPDF project, the development focus shifted on writing a new modern graphics library called “Fitz”. Fitz was originally intended as an R&D project to replace the aging Ghostscript graphics library, but has instead become the rendering engine powering MuPDF.” - [Wikipedia](https://en.wikipedia.org/wiki/MuPDF)

# Using Fitz to open PDF

In [202]:
pdf_path = '../data/ZIMMER-BIOMET_PERI-ARTICULATION.pdf'
fitz_open_pdf = fitz.open(pdf_path)

In [134]:
type(fitz_open_pdf)

fitz.fitz.Document

In [135]:
len(fitz_open_pdf)

18

In [136]:
fitz_open_pdf[1]

page 1 of ../data/ZIMMER-BIOMET_PERI-ARTICULATION.pdf

In [137]:
print(fitz_open_pdf[1])

page 1 of ../data/ZIMMER-BIOMET_PERI-ARTICULATION.pdf


---------
I found the solution below ``page.getText()` *somewhere*.....but looking back at the documentation, it says to use a different version: `page.get-text('text)`. So, I don't know where this came from....later I will make a run through of following the documentation more closely. 

But for the moment this works....

--------

In [143]:
# WE SEE THE LENGTH OF THIS LIST IS THE SAME AS THE 'fitz_open_pdf' and it has created a list.
text = [page.getText('text') for page in fitz_open_pdf]
print(type(text))
len(text)

<class 'list'>


18

In [139]:
text[5]

'Ordering\tInformation\n   \n3.5mm Proximal Lateral Humeral Locking Plate \n \nLeft \nRight \nDescription \nHoles \nLength\n\t00-2358-002-04\t\t\n00-2358-001-04\t\t\nProximal\tLateral\tHumeral\tLocking\t\t\nPlate\t4\t\t\n90mm\n\t00-2358-002-06\t\t\n00-2358-001-06\t\t\nProximal\tLateral\tHumeral\tLocking\t\t\nPlate\t6\t\t\n114mm\n\t00-2358-002-08\t\t\n00-2358-001-08\t\t\nProximal\tLateral\tHumeral\tLocking\t\t\nPlate\t8\t\t\n138mm\n\t00-2358-002-12\t\t\n00-2358-001-12\t\t\nProximal\tLateral\tHumeral\tLocking\t\t\nPlate\t12\t\t\n186mm\n\t00-2358-002-16\t\t\n00-2358-001-16\t\t\nProximal\tLateral\tHumeral\tLocking\t\t\nPlate\t16\t\t\n234mm\nPrOxIMaL\tLaTeraL\tHuMeraL\tLOckIng\tPLaTeS\nProximal Lateral Humeral Locking Plate Set \n \n \nQty\n00-2358-000-05 \nProximal Lateral Humeral Locking  \n \nPlate Set (Contains the following) \n \n \n00-2358-002-04\t\nProximal\tLateral\tHumeral\tLocking\tPlate\t \t\n\t\n1\n00-2358-002-06\t\nProximal\tLateral\tHumeral\tLocking\tPlate\t \t\n\t\n1\n00-2358

In [140]:
# Here we are extracting ONLY the pages that contain tables.
pages_of_text = text[5:-1]

In [144]:
len(pages_of_text)

12

--------
This was a see the printed output of each page....output not shown for brevity's sake.......

In [ ]:
for count, page in enumerate(pages_of_text[0:1]):
    print('-------------------------------------')
    print(f"Page{count}")
    
    print('-------------------------------------')
    print(page)
    print('-------------------------------------')

-------

In [145]:
type(pages_of_text)

list

In [146]:
type(pages_of_text[0])

str

This is a brute attempt to see what happens....

In [151]:
zimmer_list_df = pd.DataFrame(pages_of_text)

In [148]:
# We see there are 12 rows which correlates to the number of pages....so each row equals one page
zimmer_list_df

,0
0,Ordering\tInformation\n \n3.5mm Proximal Lat...
1,Item Number \nDescription \nQty\n00-2359-030-3...
2,Ordering\tInformation\t(continued)\n \nDISTa...
3,Item Number \nDescription \nQty\n00-2359-008-2...
4,Ordering\tInformation\t(continued)\n \nDISTa...
5,Item Number \nDescription \nQty\n00-2359-008-2...
6,Ordering\tInformation\t(continued)\n \n5.5mm...
7,5.5mm/4.5mm Locking Screw Set \nItem Number \n...
8,Ordering\tInformation\t(continued)\n \nDISTa...
9,Item Number \nDescription \nQty\n00-2359-030-3...


Pandas takes a 2D array....so it is convenient to have the information as a list of lists....just straight text isn't going to get us anything

In [149]:
df_page_1 = pd.DataFrame(pages_of_text[0])

ValueError: DataFrame constructor not properly called!



If we can clean this up a bit maybe it would go into a `df`, though....

----------

# Fitz: what the documentation says

It seems that in order to extract text, the docs tells you to create a text file with `sys` into which the text you extract will go.

    import sys, fitz
    fname = sys.argv[1]  # get document filename
    doc = fitz.open(fname)  # open document
    out = open(fname + ".txt", "wb")  # open text output
    for page in doc:  # iterate the document pages
        text = page.get_text().encode("utf8")  # get plain text (is in UTF-8)
        out.write(text)  # write text of page
        out.write(bytes((12,)))  # write page delimiter (form feed 0x0C)
    out.close()

**!!! First !!!**
<br>I don't understand `sys.argv[1]`

In [96]:
print(sys.argv, len(sys.argv))

['/usr/local/lib/python3.8/site-packages/ipykernel_launcher.py', '-f', '/Users/claytonlouden/Library/Jupyter/runtime/kernel-978be9f7-6ada-4847-988f-8297c941dd0e.json'] 3


In [97]:
print(sys.argv[1])

-f


[Here](https://stackoverflow.com/questions/4117530/sys-argv1-meaning-in-script) is a very good explanation of `sys.argv` on (of all places) stackoverflow.....I wish there were more answers like this.....


But after a few attempts at using `fname = sys.argv[1]` it only ended up naming the file `-f.text`.... not very useful and I am more confused why this was step was included.....

-------

### Here we have created the text file

Have decided that the `sys.argv[1]` step is not useful in this case.

In [130]:
doc = fitz.open(pdf_path)  # open the pdf document
out = open("../data/" + "zimmer" + ".txt", "wb")  # open text output into ../data/
for page in doc:  # iterate the document pages
    t = page.get_text().encode("utf8")  # get plain text (is in UTF-8)
    out.write(t)  # write text of page
    out.write(bytes((12,)))  # write page delimiter (form feed 0x0C)
out.close()

In [131]:
zimmer_text=open("../data/zimmer.txt",'r')
read_text = zimmer_text.read()

In [132]:
# This returns a long string of 62,673 chracters
print(len(read_text))
read_text[:5000]

62673


'With the Periarticular Locking Plate System, Zimmer offers advanced solutions for \nthe management of comminuted fractures and fractures in which deficient bone \nstock or poor bone quality is encountered. The system combines anatomically \ncontoured periarticular plates with locking screw technology. The unique plate \ndesigns feature a transitioned profile that is thinner in the metaphyseal and  \nthicker in the diaphyseal areas. Locking screw holes  \nare combined with compression slots,  \nso that the plates can be used  \nboth as locking devices and for  \nfracture compression.\nAdvanced fabricating technologies are guided \nby data from a digital laser scanner that helps \nto determine optimal implant contours.\nPeriarticular Locking Plate System\nCombining anatomic fit with locking technology\n Precontoured plates that follow  \nthe shape of the bone \nOne of the most difficult challenges in the treatment \nof periarticular fractures is bending the plate to fit \nthe specific p

-------
# Let's take a legitmate stab at cleaning

These are some of the characters we have to clean up.
- \b is backspace
- \n is newline
- \r is carriage return (return to left margin)
- \t is tab

We are also going to use the `list` of `lists` instead of the single `string` because we can easily choose which pages to focus on which cuts out any work to try to find the beginning of the tabular info.

**Remember** the items of the `lists` we wanted were saved to `pages_of_text`

In [181]:
drop_characters = [" \n", " \n ", "\n", "\t"]

In [182]:
page_one = pages_of_text[0]

In [183]:
for char in drop_characters:
    if char in page_one:
        rp = page_one.replace(char,'')
        sp = page_one.split(char)

In [191]:
# '\s' finds all whitespace characters
clean_one = re.sub(r'\s', ' ',page_one)

In [192]:
clean_one

'Ordering Information     3.5mm Proximal Lateral Humeral Locking Plate    Left  Right  Description  Holes  Length  00-2358-002-04   00-2358-001-04   Proximal Lateral Humeral Locking   Plate 4   90mm  00-2358-002-06   00-2358-001-06   Proximal Lateral Humeral Locking   Plate 6   114mm  00-2358-002-08   00-2358-001-08   Proximal Lateral Humeral Locking   Plate 8   138mm  00-2358-002-12   00-2358-001-12   Proximal Lateral Humeral Locking   Plate 12   186mm  00-2358-002-16   00-2358-001-16   Proximal Lateral Humeral Locking   Plate 16   234mm PrOxIMaL LaTeraL HuMeraL LOckIng PLaTeS Proximal Lateral Humeral Locking Plate Set      Qty 00-2358-000-05  Proximal Lateral Humeral Locking     Plate Set (Contains the following)      00-2358-002-04  Proximal Lateral Humeral Locking Plate      1 00-2358-002-06  Proximal Lateral Humeral Locking Plate      1 00-2358-002-08  Proximal Lateral Humeral Locking Plate      1 00-2358-002-12  Proximal Lateral Humeral Locking Plate      1 00-2358-002-16  Proxim

In [209]:
split_one = re.split(r"\s{1,}",clean_one)

In [212]:
split_one = split_one[2:]

In [216]:
split_one[:50]

['3.5mm',
 'Proximal',
 'Lateral',
 'Humeral',
 'Locking',
 'Plate',
 'Left',
 'Right',
 'Description',
 'Holes',
 'Length',
 '00-2358-002-04',
 '00-2358-001-04',
 'Proximal',
 'Lateral',
 'Humeral',
 'Locking',
 'Plate',
 '4',
 '90mm',
 '00-2358-002-06',
 '00-2358-001-06',
 'Proximal',
 'Lateral',
 'Humeral',
 'Locking',
 'Plate',
 '6',
 '114mm',
 '00-2358-002-08',
 '00-2358-001-08',
 'Proximal',
 'Lateral',
 'Humeral',
 'Locking',
 'Plate',
 '8',
 '138mm',
 '00-2358-002-12',
 '00-2358-001-12',
 'Proximal',
 'Lateral',
 'Humeral',
 'Locking',
 'Plate',
 '12',
 '186mm',
 '00-2358-002-16',
 '00-2358-001-16',
 'Proximal']

**COMMENTS**

This is really clean and quick.....but....a time consuming weeding out of the details. It is better organized than the PyPDF2.

